In [1]:
from pandas import *
from os import listdir, getcwd
from os.path import join
from json import loads
from nltk import *
from nltk import download
from nltk.corpus import stopwords
from datetime import date, datetime
import pickle
from re import *
download('stopwords')
download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df_scores = read_csv('tim_scores.csv')[['word', 'score']]

In [3]:
STOP_WORDS = stopwords.words('english')

percentile = df_scores[abs(df_scores['score'] < .8)]['score'].quantile([.25, .5, .75, 1])

final_df = DataFrame(columns=['id', 1, 2, 3, 4])

In [4]:
def append_final(df1):
    global final_df
    for index in range(len(df1)):
        try:
            q1 = []
            q2 = []
            q3 = []
            q4 = []
            total_match_count = 0

            df_row = df1.iloc[index]

            cleaned_text = ' '.join([x for x in sub('(\\|&)', ' ', sub("(,|;|!|@|#|\?|\*)", '', df_row['body'].lower())).split(' ') if x.strip('.') not in STOP_WORDS])

            text = ' '.join([x for x in cleaned_text.split('.')])

            for phrase, score in df_scores.values:
                if isinstance(phrase, str) and phrase in text and (isinstance(score, float) or isinstance(score, int)):
                    cnt = text.count(phrase)

                    if score <= percentile[.25]:
                        q1.extend([(phrase, score)] * cnt)
                    elif score <= percentile[.5]:
                        q2.extend([(phrase, score)] * cnt)
                    elif score <= percentile[.75]:
                        q3.extend([(phrase, score)] * cnt)
                    else:
                        q4.extend([(phrase, score)] * cnt)

                    total_match_count += cnt

            final_df = final_df.append({'id': df1.iloc[index]['id'], 1: len(q1) / total_match_count, 2: len(q2) / total_match_count, 3: len(q3) / total_match_count, 4: len(q4) / total_match_count, 'earnings_word_count': df1.iloc[index]['earnings_word_count'], 'quarter': df1.iloc[index]['quarter']}, ignore_index=True)
        except: print('Failed to load a earning report')
            
    return final_df

In [5]:
dir_ = 'resources/competition_test_dataset/'
comp_files = [x for x in listdir(dir_) if '.json' in x]

WORD_THRESHOLD = 20
DATAFRAME_COLUMNS = ['id', 'datetime', 'date', 'year', 'quarter', 'body', 'earnings_word_count']

df_all = DataFrame(columns=DATAFRAME_COLUMNS)

def cut_word_threshold(lst, threshold):
    return [x for x in lst if len(x.split()) >= threshold]

for file in comp_files:
    try:
        input_df = read_json(dir_ + file)[['date', 'body']]
        input_df.reset_index(inplace=True)
        input_df.columns = ['id', 'datetime', 'body']

        # Convert to datetime.date
        input_df['date'] = input_df['datetime'].apply(lambda x: x.date())
        # Convert to datetime.date
        input_df['date'] = input_df['date'].apply(lambda x: date(*[int(y) for y in str(x).split('-')]))

        input_df['year'] = input_df['date'].apply(lambda x: x.year)
        input_df['quarter'] = input_df['date'].apply(lambda x: 1 if x.month <= 3 else 2 if x.month <= 6 else 3 if x.month <= 9 else 4)

        input_df['paragraphs'] = input_df['body'].apply(lambda lst: cut_word_threshold(lst, WORD_THRESHOLD))
        input_df['body'] = input_df['paragraphs'].apply(lambda lst: ' '.join(lst))

        input_df['earnings_word_count'] = input_df['body'].apply(lambda x: len(x.split()))

        input_df = input_df[DATAFRAME_COLUMNS]

        append_final(input_df)
    except: print('Failed to load', file)

In [6]:
final_df['id'] = final_df['id'].apply(lambda x: int(x))
final_df['earnings_word_count'] = final_df['earnings_word_count'].apply(lambda x: int(x))
final_df['Q1'] = final_df['quarter'].apply(lambda x: 1 if x == 1 else 0)
final_df['Q2'] = final_df['quarter'].apply(lambda x: 1 if x == 2 else 0)
final_df['Q3'] = final_df['quarter'].apply(lambda x: 1 if x == 3 else 0)
final_df['Q4'] = final_df['quarter'].apply(lambda x: 1 if x == 4 else 0)
final_df = final_df[['id', 1, 2, 3, 4, 'earnings_word_count', 'Q1', 'Q2', 'Q3', 'Q4']]
final_df.columns = ['id', 'level 1', 'level 2', 'level 3', 'level 4', 'word_count', 'Q1', 'Q2', 'Q3', 'Q4']

In [7]:
final_df

,id,level 1,level 2,level 3,level 4,word_count,Q1,Q2,Q3,Q4
0,1299,0.405840,0.114081,0.175204,0.304875,11024,0,0,1,0
1,1300,0.411901,0.133891,0.163877,0.290330,11946,1,0,0,0
2,1301,0.392867,0.132937,0.179411,0.294785,9642,0,1,0,0
3,1303,0.404466,0.132200,0.148186,0.315148,11476,0,0,0,1
4,1306,0.407861,0.131189,0.149566,0.311383,10987,0,0,1,0
5,1307,0.399259,0.136543,0.158025,0.306173,12088,0,0,0,1
6,1308,0.413859,0.141723,0.139076,0.305342,12694,1,0,0,0
7,1309,0.414898,0.116490,0.148320,0.320292,13858,0,1,0,0
8,1310,0.415777,0.125749,0.142671,0.315803,11569,0,0,1,0
9,1311,0.410593,0.130643,0.150063,0.308701,11897,0,0,0,1


In [8]:
final_df.to_csv(dir_ + 'model_input.csv', index=False)